In [114]:
#from keras.models import load_model
from tensorflow.keras.models import load_model
from keras.initializers import glorot_uniform
from tensorflow.keras.preprocessing.image import img_to_array
from keras.utils import CustomObjectScope
import cvlib as cv
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [37]:
link='Downloads/model_emotion.h5'
model=load_model(link)

In [38]:
model.summary()

Model: "DCNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 64)        1664      
_________________________________________________________________
batchnorm_1 (BatchNormalizat (None, 48, 48, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 64)        102464    
_________________________________________________________________
batchnorm_2 (BatchNormalizat (None, 48, 48, 64)        256       
_________________________________________________________________
maxpool2d_1 (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 128)       73856  

In [39]:
mapper={0:'Anger', 1:'Fear', 2:'Happiness', 3:'Sadness', 4:'Surprise', 5:'Neutral'}

# To process through camera

In [113]:
vd=cv2.VideoCapture(0)
while vd.isOpened():
    ret,frame=vd.read()
    face, confidence=cv.detect_face(frame)
    gray=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    for idx, f in enumerate(face):
        start_x,start_y,end_x,end_y=f[0],f[1],f[2],f[3]
        #cv2.rectangle(frame,(start_x,start_y),(end_x,end_y),(0,255,0),2)
        face_crop=gray[start_y:end_y,start_x:end_x]
        face_crop = cv2.resize(face_crop, (48,48))
        face_crop = face_crop.astype("float") / 255.0
        face_crop=face_crop.reshape((1,48,48,1))
        label=mapper[model.predict_classes(face_crop)[0]]
        y=start_y-10 if start_y-10>10 else start_y +10
        if label == 'Anger':
            cv2.putText(frame, label, (start_x, y),  cv2.FONT_HERSHEY_SIMPLEX,0.7, (0, 0, 255), 2)
            cv2.rectangle(frame,(start_x,start_y),(end_x,end_y),(0,0,255),2)
        elif label=='Happiness':
            cv2.putText(frame, label, (start_x, y),  cv2.FONT_HERSHEY_SIMPLEX,0.7, (147,20,255), 2)
            cv2.rectangle(frame,(start_x,start_y),(end_x,end_y),(147,29,255),2)
        else:    
            cv2.putText(frame, label, (start_x, y),  cv2.FONT_HERSHEY_SIMPLEX,0.7, (0, 255, 0), 2)
            cv2.rectangle(frame,(start_x,start_y),(end_x,end_y),(0,255,0),2)
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF ==ord('q'):
        break
vd.release()
cv2.destroyAllWindows()
    